# Landlab and PyVista: further examples

*(Greg Tucker, University of Colorado Boulder, USA, July 2025)*

This notebook presents more extensive examples of visualizing Landlab output using PyVista. It builds on two previous notebooks, which should be reviewed first: *Translating a Landlab RasterModelGrid into a PyVista StructuredGrid for visualization* and *Translating a Landlab non-raster grid into PyVista for visualization*.

some things on the to-do list:
- hex grids
- framed voronoi grids
- animation
- flow routing
- drainage nets as lines
- sea level
- multiple surfaces with transparency
- vectors

what examples would be good?
- landscape evolution with carbonate accumulation and layers: good for SL, multiple layers, flow routing, drainage net
- submarine diffuser making a shoreline notch and perhaps terraces?
- tidal flow (hex): good for SL, vectors/quivers, coloration by water depth
- groundwater (works w/ hex?): good for viewing wt and terrain simultaneously
- mwr: good for viewing the deposit and original terrain underneath
- rfd: good for water surface and bed and shear stress (but hex or fvg?)
- lke: good for viz of fault below terrain

## Example of a listric normal fault

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyvista as pv

from landlab import HexModelGrid, imshow_grid
from landlab.components import ListricKinematicExtender
from llpvtools import grid_to_pv

In [ ]:
# parameters
nrows = 31
ncols = 51
dx = 1000.0  # grid spacing, m
nsteps = 20  # number of iterations
dt = 2.5e5  # time step, y
extension_rate = 0.001  # m/y
detachment_depth = 10000.0  # m
fault_dip = 60.0  # fault dip angle, degrees
fault_x0 = 10000.0  # m from left side of model
fault_strike = 60.0  # strike, degrees ccw from east
period = 15000.0  # period of sinusoidal variations in initial topography, m
ampl = 500.0  # amplitude of variations, m

In [ ]:
# Create grid and elevation field
grid = HexModelGrid((nrows, ncols), spacing=dx, node_layout="rect")

elev = grid.add_zeros("topographic__elevation", at="node")
elev[:] = (
    ampl
    * np.sin(2 * np.pi * grid.x_of_node / period)
    * np.sin(2 * np.pi * grid.y_of_node / period)
)

# Instantiate component
extender = ListricKinematicExtender(
    grid,
    extension_rate_x=extension_rate,
    extension_rate_y=0.0,
    fault_dip=fault_dip,
    fault_strike=fault_strike,
    fault_x0=fault_x0,
    fault_y0=0.0,
    detachment_depth=detachment_depth,
)

# Run
for i in range(nsteps):
    extender.run_one_step(dt)

In [ ]:
topomeshn, _ = grid_to_pv(grid, field_for_node_z=elev, field_for_corner_z=0.0)

In [ ]:
faultmeshn, _ = grid_to_pv(
    grid, field_for_node_z="fault_plane__elevation", field_for_corner_z=0.0
)

In [ ]:
topomeshn.set_active_scalars("topographic__elevation")
faultmeshn.set_active_scalars("fault_plane__elevation")
fault_elev = grid.at_node["fault_plane__elevation"]
cmin = np.amin(fault_elev)
cmax = np.amax(fault_elev)

pvp = pv.Plotter()
pvp.add_mesh(topomeshn, opacity=0.6, clim=[cmin, cmax], show_edges=True, cmap='autumn')
pvp.add_mesh(faultmeshn, opacity=0.6, show_edges=True, cmap='autumn')
pvp.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from landlab import HexModelGrid, imshow_grid
from landlab.components import ListricKinematicExtender
from llpvtools import grid_to_pv

In [ ]:
# Grid parameters
nrows = 260
ncols = 150
dx = 4.0
elev_name = "tidal_example_elevs_hex260x150rect_4m.txt"

# Set tidal flow parameters (these are from the MarshMorpho2D source code)
tidal_period = 12.5 * 3600.0  # tidal period in seconds
tidal_range = 3.1  # tidal range in meters
roughness = 0.02  # Manning's n
mean_sea_level = 0.0  # mean sea level in meters
min_water_depth = (
    0.01  # minimum depth for water on areas higher than low tide water surface, meters
)
nodata_code = 7.5  # code for a DEM cell with no valid data

In [ ]:
# Create grid
tidal_grid = HexModelGrid((260, 150), spacing=4.0, node_layout="rect")
z = tidal_grid.add_zeros("topographic__elevation", at="node")

# Read the elevations
z[:] = np.loadtxt(elev_name)

# Configure boundaries: any nodata nodes, plus any nodes higher than mean high tide
tidal_grid.status_at_node[z == nodata_code] = tidal_grid.BC_NODE_IS_CLOSED
tidal_grid.status_at_node[z > 1.8] = tidal_grid.BC_NODE_IS_CLOSED
boundaries_above_msl = np.logical_and(
    tidal_grid.status_at_node == tidal_grid.BC_NODE_IS_FIXED_VALUE, z > 0.0
)
tidal_grid.status_at_node[boundaries_above_msl] = tidal_grid.BC_NODE_IS_CLOSED

imshow_grid(tidal_grid, z)

In [ ]:
# Instantiate a TidalFlowCalculator component
tfc = TidalFlowCalculator(
    tidal_grid,
    tidal_period=tidal_period,
    tidal_range=tidal_range,
    roughness=roughness,
    mean_sea_level=mean_sea_level,
    min_water_depth=min_water_depth,
)

# Calculate tidal flow
tfc.run_one_step()

print(tidal_grid.at_node.keys())
print(tidal_grid.at_cell.keys())

In [ ]:
tgridn, tgridc = grid_to_pv(tidal_grid, field_for_node_z=z, field_for_corner_z=0.0)

In [ ]:
tidal_grid.at_node.keys()

In [ ]:
tgridn.set_active_scalars("mean_water__depth")
tgridn.plot()

From here:

- Translate to PyVista objects for:
  - Elevation of ground
  - Water surface elevation
- Make a PyVista plot that shows both the elevations and the water surface (with some transparency)
- Try mapping vectors to nodes and plotting quivers

In [ ]:
z[z==10.0] = 0.0
print(np.amax(z))

In [ ]:

# Instantiate a TidalFlowCalculator component
tfc = TidalFlowCalculator(
    grid,
    tidal_period=tidal_period,
    tidal_range=tidal_range,
    roughness=roughness,
    mean_sea_level=mean_sea_level,
    min_water_depth=min_water_depth,
)

# Calculate tidal flow
tfc.run_one_step()

# make plots...
plot_tidal_flow(grid, resample=5)